In [1]:
%reload_ext autoreload
%autoreload 2

from random import gauss
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
import pandas as pd
from ipywidgets import FloatSlider, interact, Layout, fixed

from least_squares import fit_line, line, LSRegression, LSClassification, LSRidgeRegression, phi_poly
from least_squares_scikit import ls_regression_scikit

rng = np.random.default_rng()
width = 9.5
plt.rcParams['figure.figsize'] = [width, width / 1.618] 
plt.rcParams['figure.dpi'] = 100

# Machine Learning Basic Module

Florian Walter, Tobias Jülg, Pierre Krack

## General Information About Implementation Assignments
We will use the Jupyter Notebook for our implementation exercises. The task description will be provided in the notebook. The code is also run in the notebook. However, the implementation itself is done in additional files which are imported in the notebook. Please do not provide any implementation that you want to be considered for correction in this notebook, but only in Python files in the marked positions. A content of a Python file could for example look similar as shown below:
```python
def f():
    ########################################################################
    # YOUR CODE
    # TODO: Implement this function
    ########################################################################
    pass
    ########################################################################
    # END OF YOUR CODE
    ########################################################################
```
To complete the exercise, remove the `pass` command and only use space inside the `YOUR CODE` block to provide a solution. Other lines within the file may not be changed in order to deliver a valid submission.

## General Information About Theory Assignments
This Jupyter Notebook also includes a theory assignment. You can either type set your solution in $\LaTeX$/Word or prepare a digital written or scanned solution.

## Assignment 2: Least Squares Regression

In this assignment you will use the least squares method for regression and classification.
We start with the very simplest form to illustrate the concept: you are given a dataset $D$ consisting of pairs $(x, y)$.
The problem we want to solve first (linear regression) is, given this dataset and a new value $x \notin D$, what is the most likely corresponding value $y$.

If you want a more concrete example, imagine the dataset contains the height ($x$) and the bone mass ($y$) for 100 unknown individuals.
Now the task is, given that dataset, how much bone mass does a two meter tall human have?

In principle we could use many different functions with different numbers of parameters to make this prediction, but we keep it simple and just use the equation of a line: $y = mx+b$.

Run the next cell to visualize a linear model and its errors.

NOTE: The next cell uses the interactive plotting capabilities of matplotlib and [ipywidgets](https://ipywidgets.readthedocs.io/en). Use it to modify $m$ and $b$. It **does not work in VSCode**. If it does not work for you, change the first line from `%matplotlib notebook` to `%matplotlib inline`, then change the last line from `data_x, data_y = interactive_plot(True)` to `data_x, data_y = interactive_plot(False)` You can then change the parameters $m$ and $b$ inside the `interactive_plot` function.

In [1]:
%matplotlib notebook
def plot(ax, data_x, data_y, m, b):
    N = len(data_x)
    ax.scatter(data_x, data_y, label="data")
    residuals = np.zeros((N, 2, 2))
    residuals[:,0,:] = np.stack((data_x, data_y), axis=1)
    residuals[:,1,:] = np.stack((data_x, m * data_x + b), axis=1)
    residual_lines = ax.add_collection(LineCollection(residuals, linestyles="dashed", colors="tab:orange", label="residuals"))
    prediction_line, = ax.plot(data_x, line(data_x, m, b), label="predictions")
    ss = np.sum(np.square(residuals[:,0,1] - residuals[:,1,1]))
    ax.legend()
    ax.set_title(f"$m={m:.2f}, y={b:.2f}, \sum_{{i=0}}^N r_i = {int(ss)}$")
    return residual_lines, prediction_line, residuals, ss


def interactive_plot(interactive: bool):
    m = 0.5
    b = 1
    N = 100
    data_x = np.arange(0, 100, 1) + rng.normal(loc=0, scale=0.2, size=100)
    data_y = 1.1 * data_x - 50.0 + rng.normal(loc=0, scale=20, size=100)
    fig = plt.figure()
    ax = fig.add_subplot()
    residual_lines, prediction_line, residuals, ss = plot(ax, data_x, data_y, m, b)
    def update(m, b):
        residuals[:,0,:] = np.stack((data_x, data_y), axis=1)
        residuals[:,1,:] = np.stack((data_x, m * data_x + b), axis=1)
        residual_lines.set_segments(residuals)
        prediction_line.set_ydata(m * data_x + b)
        ss = np.sum(np.square(residuals[:,0,1] - residuals[:,1,1]))
        ax.set_title(f"$m={m:.2f}, y={b:.2f}, \sum_{{i=0}}^N r_i = {int(ss)}$")
    layout = Layout(width="900px")
    if interactive:
        interact(
            update,
            m = FloatSlider(min=-1.5, max=1.5, step=.001, value=m, layout=layout),
            b = FloatSlider(min=-80, max=80, step=0.001, value=b, layout=layout),
        )
    return data_x, data_y
data_x, data_y = interactive_plot(True)

<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
C:\Users\sayee\AppData\Local\Temp\ipykernel_15168\3116917072.py:12: SyntaxWarning: invalid escape sequence '\s'
  ax.set_title(f"$m={m:.2f}, y={b:.2f}, \sum_{{i=0}}^N r_i = {int(ss)}$")
C:\Users\sayee\AppData\Local\Temp\ipykernel_15168\3116917072.py:31: SyntaxWarning: invalid escape sequence '\s'
  ax.set_title(f"$m={m:.2f}, y={b:.2f}, \sum_{{i=0}}^N r_i = {int(ss)}$")
C:\Users\sayee\AppData\Local\Temp\ipykernel_15168\3116917072.py:12: SyntaxWarning: invalid escape sequence '\s'
  ax.set_title(f"$m={m:.2f}, y={b:.2f}, \sum_{{i=0}}^N r_i = {int(ss)}$")
C:\Users\sayee\AppData\Local\Temp\ipykernel_15168\3116917072.py:31: SyntaxWarning: invalid escape sequence '\s'
  ax.set_title(f"$m={m:.2f}, y={b:.2f}, \sum_{{i=0}}^N r_i = {int(ss)}$")


NameError: name 'np' is not defined

As you may have already guessed, we will now do the "learning" part of machine learning, by automatically finding the best values for $m$ and $b$.

If we make the length of the orange dotted lines (the residuals) as small as possible we find the so-called best-fit line.
Note that we cannot just minimize the sum of the residuals because the residuals can be negative: we want the sum of the residuals to go towards zero and not towards $-\infty$.

So we want to miminize *the length* of the red dotted lines, i.e. the asolute values of the residuals:
$$\sum_{i=0}^{\texttt{N}} |r_i|\textrm{,}$$
where $r_i = y_i - m \cdot x_i + b$ is the difference between the actual and the predicted data point (the error, in linear regression a.k.a. the residuals), and $N$ is the number of samples in our dataset.

How do we do that?
Typically, and you will notice that this is a pattern in machine learning, we want to take the derivative of our error (here the residuals) and set it equal to zero.
That is, find the point where the error is minimized.
Great, so we just want to derive $\sum_{i=0}^{N} |r_i|$ with respect to the parameters of our model, $m$ and $b$.
Unfortunately the absolute function is not differentiable, so we need to find another function such that, when the output of the function is minimized then the input is close to zero.
A simple function that achieves this is the square function:
$$f(x): \mathbb{R} \mapsto \mathbb{R}_{+} = x^2$$

We can now derive with respect to the parameters of the model.
When we write $\sum$ we mean $\sum_{i=0}^N$.
Try to follow what we do: it is, very simple math that you all know.
Don't be afraid by the length of it, it is just because we wrote out every single step, other online resources compress many of these steps.
$$
\begin{split}
&\frac{\textrm{d}}{\textrm{d}m} \sum \big(y_i - (mx_i + b)\big)^2 = 0 \\
\textrm{(chain-rule)} \Leftrightarrow\,&\sum 2\,(y_i - mx_i - b)(-x_i) =0 \\
\Leftrightarrow\,&\sum (y_i - mx_i - b)\,x_i = 0 \\
\Leftrightarrow\,&\sum y_ix_i - mx_i^2 - bx_i = 0 \\
\Leftrightarrow\,&m\sum x_i^2 b\sum x_i = \sum x_iy_i
\end{split}
\qquad
\begin{split}
&\frac{\textrm{d}}{\textrm{d}b} \sum \big(y_i - (mx_i + b)\big)^2 = 0 \\
\textrm{(chain-rule)}\Leftrightarrow\,&\sum 2\,(y_i - mx_i - b)(-1) = 0 \\
\Leftrightarrow\,&\sum y_i - mx_i - b = 0 \\
\Leftrightarrow\,&\sum y_i -m\sum x_i -Nb = 0 \\
\Leftrightarrow\,&m\sum x_i + bN = \sum y_i
\end{split}
$$
Now we our two equations have a nice form for solving in a system of linear equations!
$$
\begin{bmatrix}
\sum x_i^2 & \sum x_i \\
\sum x_i & N
\end{bmatrix} \cdot
\begin{bmatrix}
m \\
b
\end{bmatrix} = 
\begin{bmatrix}
\sum x_iy_i\\
\sum y_i
\end{bmatrix}\\
\begin{bmatrix}
m \\
b
\end{bmatrix} = 
\begin{bmatrix}
\sum x_i^2 & \sum x_i \\
\sum x_i & N
\end{bmatrix}^{-1} \cdot
\begin{bmatrix}
\sum x_iy_i\\
\sum y_i
\end{bmatrix}
$$
Using the formula $\begin{bmatrix}a & b \\ c & d\end{bmatrix}^{-1} = \frac{1}{ad - bc}\begin{bmatrix}d & -b\\-c&a\end{bmatrix}$, we have:

$$
\begin{bmatrix}
m \\ b
\end{bmatrix} = \frac{1}{\sum x_i^2 n - \sum x_i \sum x_i}
\begin{bmatrix}
N & -\sum x_i \\
-\sum x_i & \sum x_i^2
\end{bmatrix}
\begin{bmatrix}
\sum x_iy_i \\ \sum y_i
\end{bmatrix}
$$
And finally we can write:
$$
m = \frac{N \sum x_iy_i - \sum x_i \sum y_i}{N\sum x_i^2-\sum x_i\sum x_i}\textrm{,}\quad
b = \frac{-\sum x_i \sum x_iy_i + \sum x_i^2 \sum y_i}{N\sum x_i^2-\sum x_i\sum x_i}
$$

> **Task 1** Open [`least_squares.py`](./least_squares.py) and implement `line()` and `fit_line()` using the formula derived above for $m$ and $b$. Then run the next cell to see whether your code works.

In [ ]:
%matplotlib inline
m, b = fit_line(data_x, data_y)
fig = plt.figure()
ax = fig.add_subplot()
residual_lines, prediction_line, residuals, ss = plot(ax, data_x, data_y, m, b)

## More Dimensions and the Ridge regression
In the last section, we handled the simple special case where a single numeric value is assigned a number (remember: predict weight $y$ based on height $x$).
Typically however, $y$ will depend on multiple values.
Additionaly the data might not follow a straight line but some polynomial curve.

Fortunately, there is a solution to this: we can derive the following error formula, set it to zero and solve for $w$:
$$E(w) = \frac{1}{2} \sum_{i=1}^{N} (w^T \phi(x_i) - y_i)^2$$
Where this time $w$ is a vector and $\phi$ is a so-called basis function.
In the simple line example above $w$ would be the vector $\begin{pmatrix}m\\b\end{pmatrix}$ and $\phi(x) = \begin{pmatrix}x\\1\end{pmatrix}$ such that $w^T\phi(x_i) = (m, b)\cdot\begin{pmatrix}x\\1\end{pmatrix} = mx+b$.
Choose $\phi(x) = \begin{pmatrix}x^2\\1\end{pmatrix}$ and you can find a good fit for a dataset shaped like this:

In [ ]:
f = lambda x: x**2
x = np.linspace(-1, 1, 100)
y = f(np.linspace(-2, 2, 100)) + np.random.random(100) * - 0.5
plt.scatter(x, y)
plt.show()

Following this scheme you can find a good fit regardless of the dimension of your dataset and regardless of its shape, proviced you choose the right basis functions.
In the slides there is the derivation for finding the error-minimizing weights in this general case.

Now it is your turn, adapt the derivation shown in class to the Ridge regression! 

> **Task 2** Follow that example to derive a closed form solution for the equation $\nabla_w E_{\textrm{ridge}}(w) = 0$
Reminder: The ridge regression error function looks like this:
$$E_{\text{ridge}}(w) = \frac{1}{2} \sum_{i=1}^{N} (w^T \phi(x_i) - y_i)^2 + \frac{\lambda}{2} w^T w$$
where $x_i\in\mathbb{R}^{D}$ is the input vector, $\phi: \mathbb{R}^D \rightarrow \mathbb{R}^M$ the basis functions, $w\in\mathbb{R}^M$ the weights and $y_i\in\mathbb{R}$ are the regression labels.
The dataset contains $N$ samples.
>
>Hint: You can use the design matrix introduced in the course: $\Phi \in\mathbb{R}^{N\times M}$
$$
\Phi = \begin{bmatrix}
\phi_1(x_1) & \phi_2(x_1) & \dots & \phi_M(x_1) \\
\vdots &\vdots & \ddots &\vdots  \\
\phi_1(x_N) & \phi_2(x_N) & \dots & \phi_M(x_N) \\
\end{bmatrix}
$$
where $\phi_j(x_i): \mathbb{R}^D \rightarrow \mathbb{R}$ is the j-th component of the $\phi$ function.
>
>The following steps may be used as a guidance for the derivation:
>1. Write down the error term for the ridge regression in matrix notation using the $\Phi$ matrix.
>2. Calculate the gradient from the error function $\nabla_w E_{\text{ridge}}(w)$. Hint: You might use Equations (69) and (81) from the [Matrixcookbook](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf) for the derivation if you are new to vector analysis.
>3. Set the gradient to zero and solve for $w$.

### Ordinary Least Squares Regression Special Case of Ridge Regression
Show that ridge regression on a design matrix $\Phi\in\mathbb{R}^{N\times M}$ with regularization strength $\lambda$ is equivalent to ordinary least squares regression with an augmented design matrix and target vector
$$
    \hat{\Phi} = \begin{pmatrix} \Phi \sqrt{\lambda} \pmb{I}_M \end{pmatrix} \quad \text{and} \quad \hat{y} = \begin{pmatrix} y \pmb{0}_M \end{pmatrix}\text{, }
$$
where $\pmb{I}_M$ is the identity matrix of size $M$ and $\pmb{0}_M$ is the zero vector of size $M$.

### Implementation
Now that we derived the second more general equation, we can implement it using efficient parallel numpy functions, which map nicely to the equations derived above
> **Task 3** Open [`least_squares.py`](./least_squares.py) and implement the missing parts of the `LSRegression`. Then look at the cell below and understand how we generate the data (in the line `z = (...)`), then go back to [`least_squares.py`](./least_squares.py) and implement an appropriate `phy_poly` function.
Using the convenient and efficient functions of numpy all the missing components can be implemented as simple one-liners.
NOTE: you are not allowed to use `np.linalg.lstsq`, neither are you allowed to use `np.linalg.pinv`.

Run the next cell to see if your implementation works. The orange surface shows the prediction of the linear regression model.

In [ ]:
reg = LSRegression(dim=2, m=3, phi=phi_poly)
x, y = np.mgrid[-10:10:5j, -10:10:5j]
z = (2*x**2 - 2*y**2 + 2.71828) + np.random.normal(scale=10, size=x.shape)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x, y, z, color="black", label="data")
data = np.stack([x.ravel(), y.ravel()], 1)
reg.fit(data, z.ravel())
x, y = np.mgrid[-15:15:25j, -15:15:25j]
ax.plot_surface(x, y, reg.predict(np.stack([x.ravel(), y.ravel()], 1)).reshape((25, 25)), alpha=0.7, cmap=cm.Oranges)
ax.legend()
plt.show()

> **Task 4** Implement the ridge regression by filling in the missing parts in the `LSRidgeRegression` class inside the [`least_squares.py`](least_squares.py) file. Then run the next cell to see the difference between the basic and the ridge regression below.

In [ ]:
n = 10
x = np.linspace(-1, 1, n).reshape((n, 1))
y = 2 * x + 1 + rng.normal(scale=0.2, size=(n,1))
y.reshape((n,1))
degree = 10
phi = lambda x: np.array([x[0]**n for n in range(degree)])

reg = LSRegression(dim=1, m=degree, phi=phi)
reg.fit(x, y)
ridgereg = LSRidgeRegression(dim=1, m=degree, phi=phi)
ridgereg.fit(x, y, lmbda=0.1)

fig = plt.figure()
ax = fig.subplots(1,2)
ax[0].scatter(x, y)
ax[1].scatter(x, y)
x = np.linspace(-1.5, 1.5, 100).reshape(100,1)
y = reg.predict(x)
ax[0].plot(x, y)
ax[0].set_ylim(-2, 4)
y = ridgereg.predict(x)
ax[1].plot(x, y)
ax[1].set_ylim(-2, 4)
plt.show()
print(f"Normal regression: w = (" + ", ".join(f"{w:.2f}" for w in reg.weights.flatten()) + ")")
print(f"Ridge regression: w = (" + ", ".join(f"{w:.2f}" for w in ridgereg.weights.flatten()) + ")")

# Least Squares Classification

Perhaps surprisingly, we can use the least squares method for classification.

> **Task 5** implement the missing parts of the `LSClassification` class inside the [`least_squares.py`](least_squares.py) file. Use the Bishop book chapter 4.1.3 for guidance. Use the next cell to plot the predictions of your linear classifier and verify if your implementation is correct. The points on the bottom left should be blue, top left green and top right orange.

In [ ]:
data = np.concatenate(
    (
        rng.normal(loc=-1, scale=0.2, size=(50, 2)),
        rng.normal(loc=1, scale=0.2, size=(50, 2)),
        np.stack(
            (
                rng.normal(loc=-1, scale=0.2, size=50),
                rng.normal(loc=1.0, scale=0.2, size=50)
            ),
            axis=1
        )
    )
)

labels = np.zeros((150, 3))
labels[:50,0] = 1
labels[50:100,1] = 1
labels[100:150,2] = 1

classifier = LSClassification(dim=2, k=3)
classifier.fit(data, labels)
colors = [("tab:blue", "tab:orange", "tab:green")[p] for p in classifier.predict(data)]
plt.scatter(data[:,0], data[:,1], color = colors)
x = classifier.augment(np.array([[-1.5, -1.5],[1.5,1.5]]))
plt.show()

# Real data and scikit-learn
Like in the first exercise we will now use real data along with [pandas](https://pandas.pydata.org) and [scikit-learn](https://scikit-learn.org) to illustrate a real world workflow. We use the recently published [weather prediction dataset](https://zenodo.org/records/7525955) specifically designed for teaching. It is "complex enough to demonstrate realistic issues such as overfitting and unbalanced data, while still remaining intuitively accessible." It contains data collected from 2000 to 2010 at the following weather stations:
![WEATHER_PREDICTION_MAP](weather_prediction_dataset_map.png)

Because the pandas data manipulation code is a bit more involved (multi indexing is part of the [advanced ](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) section of the user guide), we will do that for you. Try to follow along. You can type `display(df)` at any point in the following code cell if you want to see the intermediate results.

In [ ]:
# Read the data and remove the "MONTH" column, we don't need it.
df = pd.read_csv("./weather_prediction_dataset.csv").drop("MONTH", axis=1)
print("Before modifying the dataframe:")
display(df.head(10))
# Transform the "DATE" column into a pandas time series object and use it as the index of the dataframe.
# This enables things like 'data.index.month' as seen below
df.index = pd.to_datetime(df.pop("DATE"), format="%Y%m%d")
# This snippet was taken from https://github.com/pandas-dev/pandas/issues/7454
# For example,'DJF' means december january february. This is apparently how seasons are often named in geosciences.
# It creates the following array: ['DJF' 'DJF' 'MAM' 'MAM' 'MAM' 'JJA' 'JJA' 'JJA' 'SON' 'SON' 'SON' 'DJF']
SEASONS = np.array(['DJF', 'MAM', 'JJA', 'SON'])
month = np.arange(12) + 1
season = SEASONS[(month // 3) % 4]
###
# Here we do multi-level indexing, we compute the mean per year and month
df = df.groupby([(df.index.year), (season[df.index.month-1])]).mean()
print("After modifying the dataframe:")
display(df.head(10))
# Now we select the mean minimal temperature in winter and mean maximal temperature in summer
min_temp_winter = df.xs("DJF", level=1).filter(regex=r".*_temp_min").mean(axis=1)
max_temp_summer = df.xs("JJA", level=1).filter(regex=r".*_temp_max").mean(axis=1)
# Reshape it for scikit
X_data = min_temp_winter.index.to_numpy().reshape(-1,1)
print("Min temp winter") 
print(min_temp_winter)
print("Max temp summer")
print(max_temp_summer)

Now we have the mean minimal temperatures in winter and the mean maximal temperatures in summer, i.e. the extreme weather in europe. You will now use this data to fit a linear regression model using [scikit-learn](https://scikit-learn.org). The you will use the fitted model to predict the weather in the future.

> **Task 6** Open [`ls_regression_scikit.py`](./ls_regression_scikit.py) and implement the function 'ls_regression_scikit' using scikit-learn.

In [ ]:
predict_until = 2050
winter_predictions = ls_regression_scikit(X=X_data, y=min_temp_winter.values, predict_until=predict_until)
summer_predictions = ls_regression_scikit(X=X_data[:-1], y=max_temp_summer.values, predict_until=predict_until)
x = np.arange(1999, predict_until, 1).reshape(-1, 1)
plt.scatter(X_data, min_temp_winter.values)
plt.plot(
    x,    
    winter_predictions,
    label="Mean min temperature in winter"
)
plt.scatter(max_temp_summer.index, max_temp_summer.values)
plt.plot(
    x,
    summer_predictions,
    label="Mean max temperature in summer"
)
plt.title("Extreme temperatures in Europe")
plt.xlabel("Year")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.show()

> **Task 7** Discuss on Canvas: Is it reasonable to predict the weather as we did above? Why / why not?
As a hint, try predicting the weather 1000 years from now.